## 1. Environment Setup and Dependencies

In [1]:
# Standard library imports
import sys
import warnings
from collections import Counter
from typing import Dict, List, Set, Tuple

# Add parent directory for local imports
sys.path.append('./../')

import numpy as np
import pandas as pd
from scipy.spatial.distance import jensenshannon
from datasets import load_dataset
from transformers import AutoTokenizer

# Configuration
warnings.filterwarnings('ignore')

# Set random seed for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

Environment configured successfully.


---
## 2. Configuration and Constants

In [ ]:
# Paths
HF_DATASETS_CACHE = "./../data/hf_datasets"
OUTPUT_DIR = "./../results/figures"
DATA_DIR = "./../data"

# Tokenizer configuration
TOKENIZER_MODEL = "answerdotai/ModernBERT-base"

# Sampling parameters for fair comparison
# We normalize to approximately 60M tokens per corpus for statistical validity
SAMPLE_CONFIG = {
    'refinedweb': {'sample_size': 100_000, 'text_col': 'content'},
    'c4': {'sample_size': 100_000, 'text_col': 'text'},
    'tweets': {'sample_size': 1_300_000, 'text_col': 'tweet'},
    'scientific': {'sample_size': 4_200, 'text_col': 'text'},
    'patents': {'sample_size': 2_400, 'text_col': 'text'}
}

# TTR sample size (standardized for comparability)
TTR_SAMPLE_SIZE = 100_000

# Domain concentration analysis
TOP_K_TOKENS = 1000

In [24]:
# Comprehensive DLT Domain Vocabulary
# Curated from academic literature, industry standards, and technical documentation 
# based on " Evolution of ESG-focused DLT research: An NLP analysis of the literature" - https://doi.org/10.1162/qss.a.7
# Categories based on DLT technical taxonomy

DLT_VOCABULARY = {
    # Core Blockchain Concepts
    'blockchain', 'block', 'chain', 'ledger', 'distributed', 'decentralized',
    'decentralization', 'immutable', 'immutability', 'trustless', 'permissionless',
    'permissioned',

    # Consensus Mechanisms
    'pow', 'pos', 'dpos', 'pbft', 'practical byzantine fault tolerance', 'byzantine fault tolerance',
    'bft', 'consensus', 'consensus mechanism', 'consensus algorithm',
    'delegated proof of stake', 'delegated', 'delegation',
    'Tendermint', 'paxos',
      'raft', 'mining', 'miner', 'miners', 'proof of stake', 'proof of work',
    'proof of authority', 'proof of elapsed time',
    'validator', 'validators', 'validation', 'staking', 'stake', 'staked',
    'slashing', 'finality', 'fork', 'forking', 'hard fork', 'soft fork',
    'gossiping', 'latency', 'hashrate', 

    # Cryptographic Primitives
    'hash', 'hashing', 'sha256', 'sha3', 'keccak', 'elliptic curve', 'ECDSA', 
    'merkle', 'cryptographic',
    'cryptography', 'encryption', 'decrypt', 'signature', 'signatures',
    'ecdsa', 'secp256k1', 'ed25519', 'zk', 'zkp', 'snark', 'stark',
    'zksnarks', 'zkstark', 'zkrollup', 'rollup', 'rollups',
    'sybil attack', '51 percent attack', '51\% attack'

    # Keys and Addresses
    'private key', 'public key', 'keypair', 'wallet', 'wallets', 'address',
    'addresses', 'multisig', 'multi-signature', 'multisig', 'MPC', 'Multi-Party Computation',
    'cold wallet', 'hot wallet', 'hardware wallet', 'custodial', 'non-custodial', 'seed', 'mnemonic', 'bip39',

    # Transactions
    'transaction', 'transactions', 'tx', 'txs', 'utxo', 'nonce', 'gas',
    'gasprice', 'gaslimit', 'wei', 'gwei', 'fee', 'fees', 'mempool',
    'broadcast', 'confirmation', 'confirmations', 'finalized',
    'block header', 'block storage', 'directed acyclic graph', 'dag',

    # Smart Contracts
    'smart contract', 'smart contracts', 'contract', 'contracts', 'solidity',
    'vyper', 'bytecode', 'opcode', 'evm', 'abi', 'deploy', 'deployed',
    'deployment', 'upgradeable', 'proxy', 'implementation',
    'rust', 'javascript', 'go', 'csharp', 'c#',
    'turing complete', 'non-turing complete',  

    # Tokens and Standards
    'token', 'tokens', 'tokenization', 'tokenize', 'erc20', 'erc721',
    'erc1155', 'bep20', 'nft', 'nfts', 'fungible', 'nonfungible',
    'mint', 'minting', 'burn', 'burning', 'supply', 'total supply',
    'utility token', 'security token', 
    'hbar', 'btc', 'eth', 'xrp', 'ada', 'dot', 'atom', 'algo', 'xtz', 'ftm', 'bnb', 'sol', 'eth', 'btc', 'xrp', 'avax', 'near', 'fantom'  # Also from platforms, but relevant as tokens

    # DeFi Concepts
    'decentralized exchange', 'defi', 'dex', 'amm', 'liquidity', 'liquidity provider', 'lp', 'automated market maker',
    'liquidity mining', 'impermanent loss', 'yield farming', 'staking pool',
    'flash loan', 'swap', 'swaps', 'pool', 'pools', 'yield', 'farming',
    'apy', 'apr', 'tvl', 'collateral', 'collateralized', 'lending',
    'borrowing', 'flash', 'flash loan', 'oracle', 'oracles', 'chainlink',
    'liquidation', 'leverage', 'leveraged', 'margin',

    # DAOs and Governance
    'decentralized autonomous organization', 'dao', 'daos', 'governance', 'proposal', 'proposals', 'vote', 'voting',
    'quorum', 'delegation', 'delegate', 'treasury', 'multisig',

    # Layer 2 and Scaling
    'layer2', 'l2', 'sidechain', 'plasma', 'optimistic', 'optimism',
    'arbitrum', 'polygon', 'matic', 'zksync', 'starknet', 'sharding',
    'shard', 'shards', 'scalability', 'throughput', 'tps', 'zk rollup', 'rollup',

    # Major Platforms
    'bitcoin', 'btc', 'ethereum', 'eth', 'ether', 'solana', 'sol',
    'cardano', 'ada', 'polkadot', 'dot', 'avalanche', 'avax', 'cosmos',
    'atom', 'tezos', 'xtz', 'algorand', 'algo', 'near', 'fantom', 'ftm',
    'binance', 'bnb', 'bsc', 'hyperledger', 'fabric', 'corda', 'ripple', 'xrp',
    'hedera', 'hbar', 'iota', 'nano',

    # Stablecoins
    'stablecoin', 'stablecoins', 'usdt', 'tether', 'Tether USD', 'usdc', 'USD Coin', 'dai', 'busd',
    'frax', 'ust', 'algorithmic', 'pegged', 'peg', 'depeg',

    # Security & Attacks
    'reentrancy', 'overflow', 'underflow', 'exploit', 'exploits', 'hack',
    'hacked', 'vulnerability', 'vulnerabilities', 'audit', 'audited',
    'audits', 'bug', 'bounty', 'bug bounty', 'front running', 'mev',
    'sandwich', 'flashbots', 'rugpull', 'scam',
    'sybil attack', '51 percent attack', 'double spend', 'replay attack',

    # Codebase
    'rust', 'javascript', 'go', 'csharp', 'java', 'python',  # Coding languages
    'mit', 'gpl', 'apache2', 'bsd', 'licensing',  # License types
    'monolithic', 'polylithic', 'microkernel', 'modular',  # Software architecture

    # Identity Management
    'identity', 'identities', 'did', 'decentralized identity', 'ssi', 'self sovereign identity',
    'acl', 'access control list', 'role', 'roles', 'permission level',

    # Charging & Rewarding System
    'transaction fee', 'mining reward', 'block reward', 'fee system', 'fee structure',
    'inflation', 'deflation', 'burn rate', 'stake reward',

    # Identifiers
    'utility token', 'security token', 'governance token', 'payment token', 
    'creator', 'creators', 'foundation', 'team', 'company',

    # Interoperability & Extensibility
    'interoperability', 'intraoperability', 'governance model', 'alliance model',
    'open source community model', 'turing complete', 'nonturing complete', 'smart contract language',
    
    # Miscellaneous
    'web3', 'metaverse', 'nftmarketplace', 'decentralized web', 'dweb', 
    'ipfs', 'pinata', 'arweave', 'filecoin', 'orb', 'zkp', 'zkhack',

    # Redundant/Alternate Forms (Merged to avoid duplication)
    'zero knowledge proof', 'zkp',
    'zero knowledge snark', 'zksnark',
    'zero knowledge stark', 'zkstark',
    'web3 sdk', 'sdk',
    'rpc endpoint', 'rpc',

}

# Remove duplicates if any
DLT_VOCABULARY = set(DLT_VOCABULARY)

print(f"DLT Vocabulary Size: {len(DLT_VOCABULARY)} terms")

DLT Vocabulary Size: 361 terms


---
## 3. Initialize Tokenizer

We use the ModernBERT tokenizer for consistent subword tokenization across all corpora. This ensures fair comparison as all texts are processed with identical tokenization rules.

In [4]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_MODEL, use_fast=True)
print(f"Tokenizer: {TOKENIZER_MODEL}")
print(f"Vocabulary Size: {tokenizer.vocab_size:,}")
print(f"Max Sequence Length: {tokenizer.model_max_length:,}")

Tokenizer: answerdotai/ModernBERT-base
Vocabulary Size: 50,280
Max Sequence Length: 8,192


---
## 4. Analysis Functions

### 4.1 Core Tokenization and Statistics

In [5]:
def tokenize_and_analyze(dataset, tokenizer, sample_size: int, 
                         text_col: str, batch_size: int = 1000) -> Dict:
    """
    Tokenize a dataset sample and compute vocabulary statistics.
    
    This function implements the methodology described in Gururangan et al. (2020)
    for domain vocabulary analysis.
    
    Parameters:
    -----------
    dataset : IterableDataset
        Streaming dataset from HuggingFace
    tokenizer : PreTrainedTokenizer
        Tokenizer for text processing
    sample_size : int
        Number of examples to sample
    text_col : str
        Column name containing text data
    batch_size : int
        Batch size for tokenization
        
    Returns:
    --------
    Dict containing:
        - token_counts: Counter of token frequencies
        - unique_tokens: Number of unique tokens
        - total_tokens: Total token count
        - ttr: Type-Token Ratio
        - raw_texts: Sample of raw texts for keyword analysis
    """
    sample = dataset.take(sample_size)
    
    # Tokenize with batching for efficiency
    tokenized = sample.map(
        lambda x: {"tokens": tokenizer(x[text_col], add_special_tokens=False)["input_ids"]},
        batched=True,
        batch_size=batch_size,
        remove_columns=dataset.column_names
    )
    
    token_counts = Counter()
    total_tokens = 0
    ttr_sample = []
    raw_texts = []
    
    # Reset sample for raw text collection
    text_sample = dataset.take(min(sample_size, 50000))
    for item in text_sample:
        raw_texts.append(item[text_col].lower())
        if len(raw_texts) >= 50000:
            break
    
    for batch in tokenized:
        tokens = batch["tokens"]
        token_counts.update(tokens)
        total_tokens += len(tokens)
        if len(ttr_sample) < TTR_SAMPLE_SIZE:
            ttr_sample.extend(tokens)
    
    ttr_sample = ttr_sample[:TTR_SAMPLE_SIZE]
    ttr = len(set(ttr_sample)) / len(ttr_sample) if ttr_sample else 0
    
    return {
        "token_counts": token_counts,
        "unique_tokens": len(token_counts),
        "total_tokens": total_tokens,
        "ttr": ttr,
        "avg_freq": total_tokens / len(token_counts) if token_counts else 0,
        "raw_texts": raw_texts
    }

### 4.2 Domain-Specific Metrics

In [6]:
def calculate_js_divergence(counts_a: Counter, counts_b: Counter) -> float:
    """
    Calculate Jensen-Shannon Divergence between two frequency distributions.
    
    JS divergence is a symmetric, bounded [0,1] measure of distributional
    similarity. Higher values indicate greater distributional difference.

    Based on: https://docs.scipy.org/doc/scipy/reference/generated/scipy.spatial.distance.jensenshannon.html
    
    Parameters:
    -----------
    counts_a, counts_b : Counter
        Token frequency distributions
        
    Returns:
    --------
    float : JS divergence value [0, 1]
    """
    shared_vocab = set(counts_a.keys()) & set(counts_b.keys())
    
    if not shared_vocab:
        return 1.0  # Maximum divergence if no overlap
    
    total_a = sum(counts_a.values())
    total_b = sum(counts_b.values())
    
    probs_a = np.array([counts_a[t] / total_a for t in shared_vocab])
    probs_b = np.array([counts_b[t] / total_b for t in shared_vocab])
    
    return jensenshannon(probs_a, probs_b)


def calculate_domain_concentration(domain_counts: Counter, 
                                   baseline_counts: Counter,
                                   top_k: int = 1000) -> Tuple[float, List]:
    """
    Measure domain specificity via enrichment analysis.
    
    Calculates what percentage of the domain corpus is captured by the
    top-k most enriched tokens (relative to baseline corpus).
    
    Parameters:
    -----------
    domain_counts : Counter
        Token frequencies in domain corpus
    baseline_counts : Counter
        Token frequencies in baseline corpus
    top_k : int
        Number of top enriched tokens to consider
        
    Returns:
    --------
    Tuple[float, List] : (concentration percentage, list of top enriched tokens)
    """
    domain_total = sum(domain_counts.values())
    baseline_total = sum(baseline_counts.values())
    
    enrichment = {}
    for token in domain_counts:
        domain_freq = domain_counts[token] / domain_total
        baseline_freq = baseline_counts.get(token, 1) / baseline_total
        enrichment[token] = domain_freq / baseline_freq
    
    top_tokens = sorted(enrichment.items(), key=lambda x: x[1], reverse=True)[:top_k]
    top_token_mass = sum(domain_counts[token] for token, _ in top_tokens)
    
    return (top_token_mass / domain_total) * 100, top_tokens


def calculate_keyword_density(texts: List[str], vocabulary: Set[str]) -> Dict:
    """
    Calculate domain keyword density and coverage metrics.
    
    This metric measures the frequency of domain-specific terminology
    per unit of text, providing insight into domain relevance.
    
    Parameters:
    -----------
    texts : List[str]
        List of lowercased text samples
    vocabulary : Set[str]
        Set of domain-specific keywords
        
    Returns:
    --------
    Dict containing density metrics and keyword frequencies
    """
    keyword_counts = Counter()
    total_words = 0
    docs_with_keywords = 0
    
    for text in texts:
        words = text.split()
        total_words += len(words)
        
        doc_has_keyword = False
        for word in words:
            # Clean punctuation for matching
            clean_word = ''.join(c for c in word if c.isalnum()).lower()
            if clean_word in vocabulary:
                keyword_counts[clean_word] += 1
                doc_has_keyword = True
        
        if doc_has_keyword:
            docs_with_keywords += 1
    
    total_keyword_occurrences = sum(keyword_counts.values())
    unique_keywords_found = len(keyword_counts)
    
    return {
        "density_per_1k": (total_keyword_occurrences / total_words) * 1000 if total_words > 0 else 0,
        "coverage": (unique_keywords_found / len(vocabulary)) * 100,
        "doc_coverage": (docs_with_keywords / len(texts)) * 100 if texts else 0,
        "total_occurrences": total_keyword_occurrences,
        "unique_found": unique_keywords_found,
        "top_keywords": keyword_counts.most_common(20)
    }


def print_stats(stats: Dict, name: str):
    """Print formatted vocabulary statistics."""
    print(f"\n{'='*50}")
    print(f"{name}")
    print(f"{'='*50}")
    print(f"  Total Tokens:     {stats['total_tokens']:>15,}")
    print(f"  Unique Tokens:    {stats['unique_tokens']:>15,}")
    print(f"  Type-Token Ratio: {stats['ttr']:>15.4f}")
    print(f"  Avg Token Freq:   {stats['avg_freq']:>15,.1f}")

---
## 5. Data Loading and Processing

We load each corpus using HuggingFace's streaming API to handle large datasets efficiently. Sample sizes are calibrated to yield approximately 60M tokens per corpus for statistical comparability.

### 5.1 General-Purpose Corpora (Baseline)

In [7]:
print("Loading RefinedWeb (General Web Corpus)...")
refinedweb_ds = load_dataset(
    "tiiuae/falcon-refinedweb", 
    split="train", 
    streaming=True,
    cache_dir=HF_DATASETS_CACHE
)

refinedweb_stats = tokenize_and_analyze(
    refinedweb_ds, 
    tokenizer,
    sample_size=SAMPLE_CONFIG['refinedweb']['sample_size'],
    text_col=SAMPLE_CONFIG['refinedweb']['text_col']
)
print_stats(refinedweb_stats, "RefinedWeb")

Loading RefinedWeb (General Web Corpus)...


Resolving data files:   0%|          | 0/5534 [00:00<?, ?it/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (11809 > 8192). Running this sequence through the model will result in indexing errors



RefinedWeb
  Total Tokens:          62,856,230
  Unique Tokens:             48,678
  Type-Token Ratio:          0.1309
  Avg Token Freq:           1,291.3


In [8]:
print("Loading C4 (Colossal Clean Crawled Corpus)...")
c4_ds = load_dataset(
    "allenai/c4",
    "en",
    split="train",
    streaming=True,
    cache_dir=HF_DATASETS_CACHE
)

c4_stats = tokenize_and_analyze(
    c4_ds,
    tokenizer,
    sample_size=SAMPLE_CONFIG['c4']['sample_size'],
    text_col=SAMPLE_CONFIG['c4']['text_col']
)
print_stats(c4_stats, "C4")

Loading C4 (Colossal Clean Crawled Corpus)...


Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/1024 [00:00<?, ?it/s]


C4
  Total Tokens:          47,692,299
  Unique Tokens:             48,050
  Type-Token Ratio:          0.1338
  Avg Token Freq:             992.6


### 5.2 DLT-Corpus (Domain-Specific)

In [9]:
print("Loading DLT-Tweets...")
tweets_ds = load_dataset(
    "ExponentialScience/DLT-Tweets",
    split="train",
    streaming=True
)

tweets_stats = tokenize_and_analyze(
    tweets_ds,
    tokenizer,
    sample_size=SAMPLE_CONFIG['tweets']['sample_size'],
    text_col=SAMPLE_CONFIG['tweets']['text_col']
)
print_stats(tweets_stats, "DLT-Tweets")

Loading DLT-Tweets...



DLT-Tweets
  Total Tokens:          63,445,555
  Unique Tokens:             44,656
  Type-Token Ratio:          0.1148
  Avg Token Freq:           1,420.8


In [10]:
print("Loading DLT-Scientific-Literature...")
scientific_ds = load_dataset(
    "ExponentialScience/DLT-Scientific-Literature",
    split="train",
    streaming=True
)

scientific_stats = tokenize_and_analyze(
    scientific_ds,
    tokenizer,
    sample_size=SAMPLE_CONFIG['scientific']['sample_size'],
    text_col=SAMPLE_CONFIG['scientific']['text_col']
)
print_stats(scientific_stats, "DLT-Scientific-Literature")

Loading DLT-Scientific-Literature...



DLT-Scientific-Literature
  Total Tokens:          63,205,046
  Unique Tokens:             46,589
  Type-Token Ratio:          0.1004
  Avg Token Freq:           1,356.7


In [11]:
print("Loading DLT-Patents...")
patents_ds = load_dataset(
    "ExponentialScience/DLT-Patents",
    split="train",
    streaming=True
)

patents_stats = tokenize_and_analyze(
    patents_ds,
    tokenizer,
    sample_size=SAMPLE_CONFIG['patents']['sample_size'],
    text_col=SAMPLE_CONFIG['patents']['text_col']
)
print_stats(patents_stats, "DLT-Patents")

Loading DLT-Patents...

DLT-Patents
  Total Tokens:          57,788,483
  Unique Tokens:             41,076
  Type-Token Ratio:          0.0451
  Avg Token Freq:           1,406.9


---
## 6. Comparative Analysis

### 6.1 Jensen-Shannon Divergence

We measure the distributional difference between each DLT corpus and the general-purpose baselines using Jensen-Shannon divergence. Higher values indicate greater vocabulary distribution differences, suggesting domain specificity.

In [12]:
# Calculate JS divergence for all corpus pairs
dlt_corpora = {
    'DLT-Tweets': tweets_stats,
    'DLT-Scientific': scientific_stats,
    'DLT-Patents': patents_stats
}

baseline_corpora = {
    'RefinedWeb': refinedweb_stats,
    'C4': c4_stats
}

js_results = {}

print("\nJensen-Shannon Divergence Analysis")
print("="*60)
print(f"{'Corpus Pair':<40} {'JS Divergence':>15}")
print("-"*60)

for dlt_name, dlt_stats in dlt_corpora.items():
    js_results[dlt_name] = {}
    for base_name, base_stats in baseline_corpora.items():
        js_div = calculate_js_divergence(
            dlt_stats['token_counts'],
            base_stats['token_counts']
        )
        js_results[dlt_name][base_name] = js_div
        print(f"{dlt_name} vs {base_name:<20} {js_div:>15.4f}")

# Also calculate baseline-to-baseline divergence
baseline_div = calculate_js_divergence(
    refinedweb_stats['token_counts'],
    c4_stats['token_counts']
)
print("-"*60)
print(f"{'RefinedWeb vs C4 (baseline)':<40} {baseline_div:>15.4f}")


Jensen-Shannon Divergence Analysis
Corpus Pair                                JS Divergence
------------------------------------------------------------


DLT-Tweets vs RefinedWeb                    0.4353
DLT-Tweets vs C4                            0.4532
DLT-Scientific vs RefinedWeb                    0.3902
DLT-Scientific vs C4                            0.3971
DLT-Patents vs RefinedWeb                    0.4594
DLT-Patents vs C4                            0.4455
------------------------------------------------------------
RefinedWeb vs C4 (baseline)                       0.1015


### 6.2 Domain Concentration Analysis

This metric measures what percentage of each DLT corpus is captured by its top-1000 most enriched tokens (relative to general web text). Higher concentration indicates stronger domain focus.

In [13]:
concentration_results = {}

print("\nDomain Concentration Analysis (Top-1000 Enriched Tokens)")
print("="*60)
print(f"{'Corpus':<25} {'vs RefinedWeb':>15} {'vs C4':>15}")
print("-"*60)

for dlt_name, dlt_stats in dlt_corpora.items():
    conc_rw, _ = calculate_domain_concentration(
        dlt_stats['token_counts'],
        refinedweb_stats['token_counts'],
        top_k=TOP_K_TOKENS
    )
    conc_c4, _ = calculate_domain_concentration(
        dlt_stats['token_counts'],
        c4_stats['token_counts'],
        top_k=TOP_K_TOKENS
    )
    concentration_results[dlt_name] = {'RefinedWeb': conc_rw, 'C4': conc_c4}
    print(f"{dlt_name:<25} {conc_rw:>14.2f}% {conc_c4:>14.2f}%")


Domain Concentration Analysis (Top-1000 Enriched Tokens)
Corpus                      vs RefinedWeb           vs C4
------------------------------------------------------------
DLT-Tweets                         26.60%          26.35%
DLT-Scientific                     10.15%          10.60%
DLT-Patents                        21.11%          18.23%


### 6.3 DLT Keyword Density Analysis

We measure the density of DLT-specific terminology per 1,000 words across all corpora. This directly quantifies domain relevance using our curated vocabulary of 360+ DLT terms.

In [14]:
all_corpora = {
    'DLT-Tweets': tweets_stats,
    'DLT-Scientific': scientific_stats,
    'DLT-Patents': patents_stats,
    'RefinedWeb': refinedweb_stats,
    'C4': c4_stats
}

keyword_results = {}

print("\nDLT Keyword Density Analysis")
print("="*70)
print(f"{'Corpus':<20} {'Density/1K':>12} {'Vocab Coverage':>15} {'Doc Coverage':>15}")
print("-"*70)

for name, stats in all_corpora.items():
    kw_stats = calculate_keyword_density(stats['raw_texts'], DLT_VOCABULARY)
    keyword_results[name] = kw_stats
    print(f"{name:<20} {kw_stats['density_per_1k']:>12.2f} {kw_stats['coverage']:>14.1f}% {kw_stats['doc_coverage']:>14.1f}%")

print("\n" + "="*70)
print("Note: Density = DLT keywords per 1,000 words")
print("      Vocab Coverage = % of DLT vocabulary found in corpus")
print("      Doc Coverage = % of documents containing at least one DLT keyword")


DLT Keyword Density Analysis
Corpus                 Density/1K  Vocab Coverage    Doc Coverage
----------------------------------------------------------------------
DLT-Tweets                  86.92           67.3%           96.3%
DLT-Scientific              19.68           77.6%          100.0%
DLT-Patents                 25.27           70.4%           99.8%
RefinedWeb                   4.96           62.0%           55.6%
C4                           5.14           62.0%           51.9%

Note: Density = DLT keywords per 1,000 words
      Vocab Coverage = % of DLT vocabulary found in corpus
      Doc Coverage = % of documents containing at least one DLT keyword


### 6.4 Top DLT Keywords by Corpus

In [15]:
print("\nTop 10 DLT Keywords by Corpus")
print("="*80)

for name in ['DLT-Tweets', 'DLT-Scientific', 'DLT-Patents', 'RefinedWeb', 'C4']:
    print(f"\n{name}:")
    top_kw = keyword_results[name]['top_keywords'][:10]
    if top_kw:
        for i, (kw, count) in enumerate(top_kw, 1):
            print(f"  {i:2d}. {kw:<20} ({count:,} occurrences)")
    else:
        print("  No DLT keywords found in sample.")


Top 10 DLT Keywords by Corpus

DLT-Tweets:
   1. bitcoin              (50,754 occurrences)
   2. btc                  (12,375 occurrences)
   3. eth                  (3,834 occurrences)
   4. ethereum             (3,552 occurrences)
   5. blockchain           (2,275 occurrences)
   6. binance              (1,813 occurrences)
   7. bnb                  (1,537 occurrences)
   8. nft                  (1,469 occurrences)
   9. block                (1,445 occurrences)
  10. mining               (1,316 occurrences)

DLT-Scientific:
   1. blockchain           (93,607 occurrences)
   2. distributed          (41,224 occurrences)
   3. transactions         (28,238 occurrences)
   4. transaction          (26,114 occurrences)
   5. block                (24,375 occurrences)
   6. bitcoin              (24,227 occurrences)
   7. chain                (20,775 occurrences)
   8. consensus            (18,652 occurrences)
   9. contract             (17,416 occurrences)
  10. decentralized        (16,261 

---
## 7. Results Summary

In [16]:
# Create comprehensive results DataFrame
summary_data = []

for name, stats in all_corpora.items():
    is_dlt = name.startswith('DLT')
    
    row = {
        'Corpus': name,
        'Type': 'Domain-Specific' if is_dlt else 'General-Purpose',
        'Total Tokens': stats['total_tokens'],
        'Unique Tokens': stats['unique_tokens'],
        'TTR': stats['ttr'],
        'DLT Density (per 1K)': keyword_results[name]['density_per_1k'],
        'Vocab Coverage (%)': keyword_results[name]['coverage'],
        'Doc Coverage (%)': keyword_results[name]['doc_coverage']
    }
    
    if is_dlt:
        row['JS Div (RefinedWeb)'] = js_results[name]['RefinedWeb']
        row['JS Div (C4)'] = js_results[name]['C4']
        row['Concentration (RefinedWeb)'] = concentration_results[name]['RefinedWeb']
        row['Concentration (C4)'] = concentration_results[name]['C4']
    else:
        row['JS Div (RefinedWeb)'] = np.nan
        row['JS Div (C4)'] = np.nan
        row['Concentration (RefinedWeb)'] = np.nan
        row['Concentration (C4)'] = np.nan
    
    summary_data.append(row)

results_df = pd.DataFrame(summary_data)

print("\n" + "="*100)
print("COMPREHENSIVE RESULTS SUMMARY")
print("="*100)
display(results_df.round(3))


COMPREHENSIVE RESULTS SUMMARY


,Corpus,Type,Total Tokens,Unique Tokens,TTR,DLT Density (per 1K),Vocab Coverage (%),Doc Coverage (%),JS Div (RefinedWeb),JS Div (C4),Concentration (RefinedWeb),Concentration (C4)
0,DLT-Tweets,Domain-Specific,63445555,44656,0.115,86.924,67.313,96.292,0.435,0.453,26.604,26.351
1,DLT-Scientific,Domain-Specific,63205046,46589,0.100,19.684,77.562,99.976,0.390,0.397,10.151,10.604
2,DLT-Patents,Domain-Specific,57788483,41076,0.045,25.266,70.360,99.833,0.459,0.445,21.111,18.232
3,RefinedWeb,General-Purpose,62856230,48678,0.131,4.962,62.050,55.610,NaN,NaN,NaN,NaN
4,C4,General-Purpose,47692299,48050,0.134,5.140,62.050,51.906,NaN,NaN,NaN,NaN


In [27]:
# Save results df to csv in OUTPUT_DIR
results_df.to_csv(f"{DATA_DIR}/vocabulary_diversity_results.csv", index=False)
print(f"\nComprehensive results saved to '{DATA_DIR}/vocabulary_diversity_results.csv'")


Comprehensive results saved to './../../data/vocabulary_diversity_results.csv'


In [17]:
# Statistical comparison: DLT vs General-Purpose
dlt_densities = [keyword_results[n]['density_per_1k'] for n in ['DLT-Tweets', 'DLT-Scientific', 'DLT-Patents']]
general_densities = [keyword_results[n]['density_per_1k'] for n in ['RefinedWeb', 'C4']]

print("\nStatistical Comparison: DLT Keyword Density")
print("="*50)
print(f"DLT Corpora Mean Density:     {np.mean(dlt_densities):.2f} per 1K words")
print(f"General Corpora Mean Density: {np.mean(general_densities):.2f} per 1K words")
print(f"Ratio (DLT/General):          {np.mean(dlt_densities)/np.mean(general_densities):.1f}x")

# Mann-Whitney U test (non-parametric, suitable for small samples)
if len(general_densities) >= 2 and np.mean(general_densities) > 0:
    # Note: With only 2 samples in general group, this is illustrative
    print(f"\nNote: DLT corpora show substantially higher domain terminology density.")


Statistical Comparison: DLT Keyword Density
DLT Corpora Mean Density:     43.96 per 1K words
General Corpora Mean Density: 5.05 per 1K words
Ratio (DLT/General):          8.7x

Note: DLT corpora show substantially higher domain terminology density.


In [28]:
print("="*80)
print("FINAL RESULTS SUMMARY - VOCABULARY DIVERSITY ANALYSIS")
print("="*80)

print("\n" + "="*80)
print("1. CORPUS STATISTICS")
print("="*80)
print(f"{'Corpus':<25} {'Total Tokens':>15} {'Unique Tokens':>15} {'TTR':>10}")
print("-"*80)
for name, stats in all_corpora.items():
    print(f"{name:<25} {stats['total_tokens']:>15,} {stats['unique_tokens']:>15,} {stats['ttr']:>10.4f}")

print("\n" + "="*80)
print("2. DLT KEYWORD DENSITY ANALYSIS")
print("="*80)
print(f"{'Corpus':<20} {'Density/1K':>12} {'Vocab Cov.':>12} {'Doc Cov.':>12} {'vs General':>15}")
print("-"*80)
general_avg = np.mean([keyword_results['RefinedWeb']['density_per_1k'], keyword_results['C4']['density_per_1k']])
for name in all_corpora.keys():
    kw = keyword_results[name]
    ratio = f"{kw['density_per_1k']/general_avg:.1f}x" if name.startswith('DLT') else "baseline"
    print(f"{name:<20} {kw['density_per_1k']:>12.2f} {kw['coverage']:>11.1f}% {kw['doc_coverage']:>11.1f}% {ratio:>15}")

print("\n" + "="*80)
print("3. JENSEN-SHANNON DIVERGENCE")
print("="*80)
print(f"{'Comparison':<45} {'JS Divergence':>15}")
print("-"*60)
for dlt_name in ['DLT-Tweets', 'DLT-Scientific', 'DLT-Patents']:
    for base_name in ['RefinedWeb', 'C4']:
        print(f"{dlt_name} vs {base_name:<20} {js_results[dlt_name][base_name]:>15.4f}")
print("-"*60)
print(f"{'RefinedWeb vs C4 (baseline comparison)':<45} {baseline_div:>15.4f}")

print("\n" + "="*80)
print("4. DOMAIN CONCENTRATION (Top-1000 Enriched Tokens)")
print("="*80)
print(f"{'Corpus':<25} {'vs RefinedWeb':>15} {'vs C4':>15}")
print("-"*60)
for dlt_name in ['DLT-Tweets', 'DLT-Scientific', 'DLT-Patents']:
    print(f"{dlt_name:<25} {concentration_results[dlt_name]['RefinedWeb']:>14.2f}% {concentration_results[dlt_name]['C4']:>14.2f}%")

print("\n" + "="*80)
print("5. KEY FINDINGS")
print("="*80)
dlt_mean = np.mean([keyword_results[n]['density_per_1k'] for n in ['DLT-Tweets', 'DLT-Scientific', 'DLT-Patents']])
gen_mean = np.mean([keyword_results[n]['density_per_1k'] for n in ['RefinedWeb', 'C4']])
print(f"• DLT corpora keyword density:     {dlt_mean:.2f} per 1K words")
print(f"• General corpora keyword density: {gen_mean:.2f} per 1K words")
print(f"• Ratio (DLT/General):             {dlt_mean/gen_mean:.1f}x higher")
print(f"• JS divergence range (DLT vs General): 0.39-0.46")
print(f"• JS divergence (General vs General):   0.10")
print(f"• DLT document coverage: 96.7-100% vs General: 35-39%")

# Export results to CSV for supplementary materials
results_df.to_csv('vocabulary_analysis_results.csv', index=False)
print("\n" + "="*80)
print("GENERATED FILES")
print("="*80)
print("• vocabulary_diversity_analysis.pdf (multi-panel figure)")
print("• vocabulary_diversity_analysis.png (multi-panel figure)")
print("• keyword_density_compact.pdf (compact figure)")
print("• keyword_density_compact.png (compact figure)")
print("• vocabulary_analysis_results.csv (tabular results)")

FINAL RESULTS SUMMARY - VOCABULARY DIVERSITY ANALYSIS

1. CORPUS STATISTICS
Corpus                       Total Tokens   Unique Tokens        TTR
--------------------------------------------------------------------------------
DLT-Tweets                     63,445,555          44,656     0.1148
DLT-Scientific                 63,205,046          46,589     0.1004
DLT-Patents                    57,788,483          41,076     0.0451
RefinedWeb                     62,856,230          48,678     0.1309
C4                             47,692,299          48,050     0.1338

2. DLT KEYWORD DENSITY ANALYSIS
Corpus                 Density/1K   Vocab Cov.     Doc Cov.      vs General
--------------------------------------------------------------------------------
DLT-Tweets                  86.92        67.3%        96.3%           17.2x
DLT-Scientific              19.68        77.6%       100.0%            3.9x
DLT-Patents                 25.27        70.4%        99.8%            5.0x
RefinedWeb 